In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,7897
2,Huelva,Confirmados PDIA 14 días,2527
3,Huelva,Tasa PDIA 14 días,"494,76938499401854"
4,Huelva,Confirmados PDIA 7 días,1168
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,1
629,Municipio de Huelva sin especificar,Total Confirmados,102
630,Municipio de Huelva sin especificar,Curados,28


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  7897.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  2172.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 465 personas en los últimos 7 días 

Un positivo PCR cada 192 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,7897.0,1168.0,2527.0,510743.0,228.686443,494.769385,167.0
Huelva-Costa,4172.0,648.0,1479.0,288115.0,224.910192,513.336688,91.0
Condado-Campiña,2479.0,390.0,718.0,155057.0,251.520409,463.055522,64.0
Huelva (capital),2172.0,309.0,750.0,143663.0,215.086696,522.055087,57.0
Lepe,648.0,103.0,238.0,27431.0,375.487587,867.631512,15.0
Sierra de Huelva-Andévalo Central,1145.0,129.0,321.0,67571.0,190.910302,475.055867,15.0
Almonte,371.0,56.0,91.0,24191.0,231.491050,376.172957,9.0
Palos de la Frontera,147.0,52.0,75.0,11289.0,460.625388,664.363540,7.0
Gibraleón,322.0,17.0,73.0,12607.0,134.845721,579.043389,7.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Corteconcepción,12.0,9.0,9.0,548.0,1642.335766,1642.335766,1.0
Aracena,277.0,22.0,104.0,8107.0,271.370421,1282.841988,2.0
Villablanca,50.0,13.0,36.0,2848.0,456.460674,1264.044944,1.0
Almendro (El),16.0,6.0,10.0,826.0,726.392252,1210.653753,0.0
Palma del Condado (La),358.0,54.0,105.0,10761.0,501.812099,975.745749,5.0
Villarrasa,86.0,15.0,20.0,2176.0,689.338235,919.117647,0.0
Lepe,648.0,103.0,238.0,27431.0,375.487587,867.631512,15.0
Campillo (El),49.0,9.0,17.0,2023.0,444.883836,840.336134,0.0
Marines (Los),5.0,1.0,3.0,396.0,252.525253,757.575758,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Villalba del Alcor,82.0,1.0,13.0,3338.0,29.958059,389.454763,1.0,0.076923
Manzanilla,44.0,1.0,7.0,2135.0,46.838407,327.868852,0.0,0.142857
Aracena,277.0,22.0,104.0,8107.0,271.370421,1282.841988,2.0,0.211538
Gibraleón,322.0,17.0,73.0,12607.0,134.845721,579.043389,7.0,0.232877
Almonaster la Real,32.0,1.0,4.0,1819.0,54.975261,219.901045,0.0,0.250000
Villanueva de los Castillejos,43.0,2.0,7.0,2820.0,70.921986,248.226950,1.0,0.285714
Zalamea la Real,41.0,2.0,7.0,3068.0,65.189048,228.161669,2.0,0.285714
Escacena del Campo,88.0,3.0,10.0,2284.0,131.348511,437.828371,0.0,0.300000
Cerro de Andévalo (El),31.0,1.0,3.0,2364.0,42.301184,126.903553,1.0,0.333333
